# Experiments with very basic magnetic field simulations
Physics informed neural networks (PINNs) 

In [ ]:
#close all matplotlib windows
import matplotlib.pyplot as plt
plt.close("all")
#clear all variables
from IPython import get_ipython
get_ipython().run_line_magic('reset', '-f')

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from numpy import ndarray
from tqdm import tqdm
from time import time

class Wire():
    def __init__(self, wire_path:ndarray, V=0.0, ρ=1.77e-8, section=1e-4, seg_len=5e-2):
        self._wp, self._V, self._ρ, self._s, self._seg_len = wire_path, V, ρ, section, seg_len
        assert self._wp.shape[1] == 3, f'wire_path must be a (n,3) array, not {self.wp.shape}'

        #calculate legnth of wire
        diff = self.wp - np.roll(self.wp, 1, axis=0) #difference between points
        self._L = np.sum(np.sqrt(np.sum(diff**2, axis=1)))

        #resample wire path with segments of length seg_len
        w = []
        for i in range(len(self.wp)):
            p1, p2 = self.wp[i], self.wp[(i+1)%len(self.wp)]
            l = np.linalg.norm(p2-p1)
            n = int(l/self.seg_len)
            for j in range(n):
                w.append(p1 + (p2-p1)*j/n)
        self._wp = np.array(w)

        self._R = self._ρ*self._L/self._s # resistance R = ρ*L/A
        self._I = self._V/self._R # current I = V/R

    def plot(self, ax, **kwargs):
        for i in range(len(self.wp)):
            p1, p2 = self.wp[i], self.wp[(i+1)%len(self.wp)]
            ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], **kwargs)
            # ax.scatter(p1[0], p1[1], p1[2], **kwargs)
        return ax

    def __str__(self) -> str:
        return f'Wire: V={self.V:.2f} V, ρ={self.ρ:.2e} Ωm, s={self.s:.2e} m^2, L={self.L:.2f} m, R={self.R:.2e} Ω, I={self.I:.2e} A'

    @property #current
    def I(self): return self._I
    @property #resistance
    def R(self): return self._R
    @property #length
    def L(self): return self._L
    @property #wire path
    def wp(self): return self._wp 
    @property #voltage
    def V(self): return self._V
    @property #resistivity
    def ρ(self): return self._ρ 
    @property #section
    def s(self): return self._s 
    @property #segment length
    def seg_len(self): return self._seg_len

def create_grid(xlim, ylim, zlim, n=(10,10,10)):
    #create a grid of points
    x = np.linspace(*xlim, n[0])
    y = np.linspace(*ylim, n[1])
    z = np.linspace(*zlim, n[2])
    grid = np.array(np.meshgrid(x,y,z)).T.reshape(-1,3)
    return grid

def create_wire_path(n=100, r=1.0, z=1.0):
    #create a wire path
    t = np.linspace(0, 2*np.pi, n+1)
    x = r*np.cos(t)
    y = r*np.sin(t)
    z = np.ones_like(x)*z
    wp = np.array([x,y,z]).T
    return wp

#define interface for MagField

class MagField():
    pass

In [ ]:
#create a path
wp = create_wire_path(n=3, r=2.0, z=0.0)

# grid = create_grid((-2,2), (-2,2), (0,2), n=(5,20,20))
grid = create_grid((-3,3), (-3,3), (-3,3), n=(15,15,5))

#create a wire
w = Wire(wp, V=50, seg_len=0.1) #create wire object

## Simple version wire + its magnetic field

In [ ]:
class MagField():
    def __init__(self, wire:Wire):
        self._w = wire
        self._B = None

    def calc_B(self, grid:ndarray):
        # calculate B field on a grid
        assert grid.shape[1] == 3, f'grid must be a (n,3) array, not {grid.shape}'
        self._B = np.zeros_like(grid) #initialize B field
        #rewrite the same loop but sobstitute the inner loop with a vectorized version
        #calculate B field
        μ0 = 4*np.pi*1e-7 #vacuum permeability
        for i, p in enumerate(tqdm(grid)):
            wp1, wp2 = self.w.wp, np.roll(self.w.wp, -1, axis=0)
            dl = wp2 - wp1  # dl|
            r = p - (wp1 + wp2) / 2  # r
            rnorm = np.linalg.norm(r, axis=1).reshape(-1,1)  # |r|
            r̂ = r / rnorm  # unit vector r
            self._B[i] += np.sum(
                μ0 * self.w.I * np.cross(dl, r̂) / (4*np.pi*rnorm**2),
                axis=0,
            )  # Biot-Savart law
        return self._B
    

    @property
    def B(self): return self._B
    @property
    def w(self): return self._w

%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
fig.suptitle('Standard numpy B field calculation')

w.plot(ax=ax, c='b') #plot wire
#plot grid
ax.scatter(grid[:,0], grid[:,1], grid[:,2], c='k', marker='.')

start_time = time()
#test MagField
mf = MagField(w)
B = mf.calc_B(grid)
print(f'\n\ncalc_B took {time()-start_time:.2f} s')

#plot B field
magnorm = np.log(1000*np.clip(np.linalg.norm(B, axis=1), 0, 0.01))
q = ax.quiver(grid[:,0], grid[:,1], grid[:,2], B[:,0], B[:,1], B[:,2], length=0.8, normalize=True, 
              color=plt.cm.viridis(magnorm), arrow_length_ratio=0.0)
plt.show()

## Symbolic calculations with sympy

In [ ]:
#install sympy
# !pip install sympy
import sympy as sp
import numpy as np
import sympy.plotting as splt
import matplotlib.pyplot as plt
from sympy import Symbol as Sym

class SymMagField():
    def __init__(self, wire:Wire):
        self._w = wire
        self._B = None

    def calc_B(self) -> sp.Matrix:
        # calculate B field symbolically
        x, y, z = sp.symbols('x y z')
        μ0 = 4*np.pi*1e-7
        B = sp.Matrix([0,0,0])
        for i in tqdm(range(len(self.w.wp))):
            wp1, wp2 = self.w.wp[i], self.w.wp[(i+1)%len(self.w.wp)]
            dl = sp.Matrix(wp2 - wp1) #dl
            r = sp.Matrix([x,y,z]) - ((wp1+wp2)/2).reshape(3,1)
            rnorm = r.norm() #|r|
            r̂ = r/rnorm # unit vector r
            B += μ0*self.w.I*dl.cross(r̂)/(4*np.pi*rnorm**2) #Biot-Savart law
        self._B = B
        #convert to numpy function
        Bnp = sp.lambdify((x,y,z), self._B, 'numpy')
        return Bnp

    @property
    def B(self): return self._B
    @property
    def w(self): return self._w

#plot 3d wire
%matplotlib
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#add title
fig.suptitle('Symbolic B field', fontsize=16)

w.plot(ax=ax, c='b') #plot wire
ax.scatter(grid[:,0], grid[:,1], grid[:,2], c='k', marker='.') #plot grid

start_time = time()
SMF = SymMagField(w) #create symbolic magnetic field object
Bnp = SMF.calc_B() #calculate B field symbolically

#calculate B field
B=np.zeros_like(grid)
for i, p in enumerate(tqdm(grid)):
    B[i] = Bnp(*p).reshape(-1) #calculate B field
print(f'\n\ncalc_B took {time()-start_time:.2f} s')

#plot B field
magnorm = np.log(1000*np.clip(np.linalg.norm(B, axis=1), 0, 0.01))
q = ax.quiver(grid[:,0], grid[:,1], grid[:,2], B[:,0], B[:,1], B[:,2], length=0.8, normalize=True, 
              color=plt.cm.viridis(magnorm), arrow_length_ratio=0.0)
plt.show()


## Plotting

In [ ]:
def plot_mag_field(field):
    pass